# Check lại xem có sót file nào không phải định dạng TXT nếu có thì đẩy qua folder error


In [1]:
import os
import shutil

def filter_output_folder(output_folder, error_folder, delete=False):
    """
    Kiểm tra thư mục output, đảm bảo chỉ có file .txt.
    Nếu có file sai định dạng:
      - `delete=True`: Xóa file sai
      - `delete=False`: Di chuyển file sai sang thư mục lỗi
    """
    os.makedirs(error_folder, exist_ok=True)
    
    non_txt_files = []
    
    for file in os.listdir(output_folder):
        file_path = os.path.join(output_folder, file)

        # Bỏ qua thư mục con nếu có
        if os.path.isdir(file_path):
            continue

        # Kiểm tra định dạng file
        if not file.lower().endswith(".txt"):
            non_txt_files.append(file_path)

    # Nếu có file sai định dạng
    if non_txt_files:
        print("\n⚠️ Các file không phải .txt trong thư mục output:")
        for file in non_txt_files:
            print(f"- {file}")

        if delete:
            # Xóa file sai
            for file in non_txt_files:
                os.remove(file)
                print(f"❌ Đã xóa: {file}")
        else:
            # Di chuyển file sai vào thư mục lỗi
            for file in non_txt_files:
                shutil.move(file, os.path.join(error_folder, os.path.basename(file)))
                print(f"⚠️ Đã di chuyển vào thư mục lỗi: {file}")
    else:
        print("✅ Thư mục output chỉ chứa file .txt, không có file sai.")

# Đường dẫn thư mục cần kiểm tra
output_folder = r"K:\convert_txt_batch_2\output_txt"  # Thư mục chứa file kết quả
error_folder = r"K:\convert_txt_batch_2\errors"  # Thư mục chứa file sai

# Chạy kiểm tra
filter_output_folder(output_folder, error_folder, delete=False)  # Đặt delete=True nếu muốn xóa file sai



⚠️ Các file không phải .txt trong thư mục output:
- K:\convert_txt_batch_2\output_txt\01_CT-BTTTT_210224131308.DOCX
- K:\convert_txt_batch_2\output_txt\163_QD-BTTTT_160224160228.DOCX
- K:\convert_txt_batch_2\output_txt\63_2023_QD-UBND_211123213434.DOC
⚠️ Đã di chuyển vào thư mục lỗi: K:\convert_txt_batch_2\output_txt\01_CT-BTTTT_210224131308.DOCX
⚠️ Đã di chuyển vào thư mục lỗi: K:\convert_txt_batch_2\output_txt\163_QD-BTTTT_160224160228.DOCX
⚠️ Đã di chuyển vào thư mục lỗi: K:\convert_txt_batch_2\output_txt\63_2023_QD-UBND_211123213434.DOC


# Chạy lại từng file doc trong folder Eror nếu fix dc thì delete và chuyển file TXT sang folder mới

In [4]:
import os
import shutil
import win32com.client
from docx import Document

def extract_text_from_doc(input_path):
    """Mở file .doc trong Microsoft Word, copy nội dung và trả về văn bản"""
    try:
        word = win32com.client.Dispatch("Word.Application")
        word.Visible = False  # Chạy Word ẩn

        doc = word.Documents.Open(input_path)
        text = doc.Content.Text  # Lấy toàn bộ nội dung văn bản
        doc.Close()
        word.Quit()
        return text.strip()
    except Exception as e:
        print(f"❌ Lỗi mở file {input_path}: {e}")
        return None

def extract_text_from_docx(input_path):
    """Mở file .docx bằng python-docx, copy nội dung và trả về văn bản"""
    try:
        doc = Document(input_path)
        text = "\n".join([para.text for para in doc.paragraphs])
        return text.strip()
    except Exception as e:
        print(f"❌ Lỗi mở file {input_path}: {e}")
        return None

def process_error_files(error_folder, fixed_folder):
    """Xử lý các file Word trong thư mục lỗi, lưu nội dung vào TXT, nếu thành công thì di chuyển TXT và xóa file Word gốc"""
    os.makedirs(fixed_folder, exist_ok=True)

    for file in os.listdir(error_folder):
        input_path = os.path.join(error_folder, file)
        
        # Kiểm tra nếu là file Word
        if file.lower().endswith(".doc"):
            output_path = os.path.join(error_folder, file.replace(".doc", ".txt"))
            extract_func = extract_text_from_doc
        elif file.lower().endswith(".docx"):
            output_path = os.path.join(error_folder, file.replace(".docx", ".txt"))
            extract_func = extract_text_from_docx
        else:
            continue  # Bỏ qua file không phải Word
        
        try:
            text = extract_func(input_path)
            if text:
                with open(output_path, "w", encoding="utf-8") as txt_file:
                    txt_file.write(text)

                # Di chuyển file TXT đã xử lý sang thư mục fixed_folder
                shutil.move(output_path, os.path.join(fixed_folder, os.path.basename(output_path)))
                print(f"✅ Đã xử lý: {input_path} -> {fixed_folder}")

                # Xóa file Word gốc nếu xử lý thành công
                os.remove(input_path)
                print(f"🗑️ Đã xóa file gốc: {input_path}")

        except Exception as e:
            print(f"❌ Lỗi xử lý {input_path}: {e}")

# Đường dẫn thư mục lỗi và thư mục lưu file TXT đã sửa
error_folder = r"K:\\convert_txt_batch_2\\errors"  # Thư mục chứa file lỗi
fixed_folder = r"K:\\convert_txt_batch_2\\fixed_output"  # Thư mục chứa file TXT sau khi sửa

# Chạy xử lý
process_error_files(error_folder, fixed_folder)


❌ Lỗi mở file K:\\convert_txt_batch_2\\errors\01_CT-BTTTT_210224131308.DOCX: Package not found at 'K:\\convert_txt_batch_2\\errors\01_CT-BTTTT_210224131308.DOCX'
❌ Lỗi mở file K:\\convert_txt_batch_2\\errors\08_vbhn_nhnn_2024_070524143424.docx: Package not found at 'K:\\convert_txt_batch_2\\errors\08_vbhn_nhnn_2024_070524143424.docx'
❌ Lỗi mở file K:\\convert_txt_batch_2\\errors\1338_tct_ttkt_080424162703.docx: Package not found at 'K:\\convert_txt_batch_2\\errors\1338_tct_ttkt_080424162703.docx'
❌ Lỗi mở file K:\\convert_txt_batch_2\\errors\163_QD-BTTTT_160224160228.DOCX: Package not found at 'K:\\convert_txt_batch_2\\errors\163_QD-BTTTT_160224160228.DOCX'
✅ Đã xử lý: K:\\convert_txt_batch_2\\errors\2249_QD_UBND_291123160546.doc -> K:\\convert_txt_batch_2\\fixed_output
🗑️ Đã xóa file gốc: K:\\convert_txt_batch_2\\errors\2249_QD_UBND_291123160546.doc
✅ Đã xử lý: K:\\convert_txt_batch_2\\errors\2250_QD_UBND_291123162301.doc -> K:\\convert_txt_batch_2\\fixed_output
🗑️ Đã xóa file gốc: K: